In [1]:
import sys
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
#from gensim.utils import lemmatize, simple_preprocess
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
from __future__ import print_function
from nltk.stem.porter import *
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD, NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import numpy as np
import os, gensim, pyLDAvis, re, nltk, spacy, string, pyLDAvis.sklearn, nltk, en_core_web_sm
import pandas as pd
%matplotlib inline
nltk.download('punkt')
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
from pyLDAvis import sklearn as sklearn_lda
import pickle 
import pyLDAvis

2022-12-14 21:48:39.526509: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/andrewwakefield2017/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
##Data Import Step
#Import Posts Data
df_p = pd.read_csv('posts_data.csv')
#Import Comments Data
df_c = pd.read_csv('comments_data.csv')

In [3]:
#Data will be prepreocessed to include only text fields

#Isolating text portions in respective dataframes
#Renaming all titles to 'text'
df_pt = df_p['title'].rename({'title': 'text'})

df_pst = df_p['selftext'].rename({'selftext': 'text'})

df_cb = df_c['body'].rename({'body': 'text'})

#Combining three dataframes
df_text = pd.concat([df_pt, df_pst, df_cb], ignore_index = True)

#Removing Duplicates and NaN values
df_text = df_text.drop_duplicates()
df_text = df_text.dropna()

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

def color_green(val):
    color = "green" if val > .1 else "black"
    return "color: {col}".format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return "font-weight: {weight}".format(weight=weight)

In [8]:
#Convert dataframe to list
data = df_text.values.tolist()

#Remove strings that will likely be too short
data = [line for line in data if len(line.split(' ')) > 18]

#Tokenize
data_words = list(sent_to_words(data))

# Initialize spacy ‘en’ model, keeping only tagger component (for efficiency)
nlp = en_core_web_sm.load(disable=['parser', 'ner'])

###function that shows top 10 words
import matplotlib.pyplot as plt
import seaborn as sns


sns.set_style('whitegrid')
# Helper function
def plot_10_most_common_words(count_data, count_vectorizer):
    import matplotlib.pyplot as plt
    words = count_vectorizer.get_feature_names()
    total_counts = np.zeros(len(words))
    for t in count_data:
        total_counts += t.toarray()[0]

    count_dict = (zip(words, total_counts))
    count_dict = sorted(count_dict, key=lambda x: x[1], reverse=True)[0:15]
    words = [w[0] for w in count_dict]
    counts = [w[1] for w in count_dict]
    
    x_pos = np.arange(len(words))

    plt.figure(2, figsize=(15, 15 / 1.6180))
    plt.subplot(title='10 most common words')
    sns.set_context("notebook", font_scale=1.25, rc={"lines.linewidth": 2.5})
    sns.barplot(x_pos, counts, palette='husl')
    plt.xticks(x_pos, words, rotation=90)
    plt.xlabel('words')
    plt.ylabel('counts')
    plt.show()

In [12]:
#Download data for classification step
df_data = pd.DataFrame(data, columns = ['text'])
df_data.to_csv('textNoClass1.csv')

In [9]:
# Do lemmatization keeping only Noun, Adj, Verb, Adverb
%time data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']) #select noun and verb

# NLTK Stop words
stop_words = stopwords.words('english')
stop_words.extend(['make', 'from', 'subject', 're', 'edu', 'use', 'people', 'going', 'even', 'just', 'still', 'really', 'many', 'thats', 'thing', 'actually', 'one', 'get' ])
stop_words.extend(['subreddit', 'im','dont','have', 'has', 'had', 'are', 'is', 'was', 'were', 'lot', 'lots', 'some', 'also', 'may', 'be'])
stop_words.extend(['question', 'ask','discuss','redditor', 'reddit', 'post','dedicate', 'discussion', 'want','would'])
stop_words.extend(['take', 'say','tell','year', 'think', 'feel'])
stop_words.extend(['com', 'http', 'amp', 'www'])

KeyboardInterrupt: 

In [17]:
df_data.iloc(5)

ValueError: No axis named 5 for object type DataFrame

In [42]:
%time vectorizer = CountVectorizer(analyzer='word', min_df=5, max_df=0.6, stop_words=stop_words, lowercase=True, token_pattern='[a-zA-Z0-9]{3,}', ngram_range=(1,2))
%time data_vectorized = vectorizer.fit_transform(data_lemmatized)
%time vocab_d = np.array(vectorizer.get_feature_names())


tfidf_vectorizer = TfidfVectorizer(analyzer='word', max_df=0.6, min_df=5, stop_words=stop_words, lowercase=True, token_pattern='[a-zA-Z0-9]{3,}', ngram_range=(1,2))

tfidf = tfidf_vectorizer.fit_transform(data_lemmatized)
tfidf_feature_names = np.array(tfidf_vectorizer.get_feature_names())



for n_components in range(5,21):
    
    
    nmf_model = NMF(n_components=n_components, random_state=30, max_iter=50, init='nndsvd', alpha=0.1 )
    nmf_output=nmf_model.fit_transform(tfidf)
    nmf_prepared=pyLDAvis.sklearn.prepare(nmf_model, tfidf, tfidf_vectorizer, sort_topics=False)
    pyLDAvis.display(nmf_prepared)
    pyLDAvis.save_html(nmf_prepared, "nmf_work"+str(n_components)+"Topics"+".html")  



    def show_topics(vectorizer=tfidf_vectorizer, lda_model=nmf_model, n_words=15):
        keywords = np.array(vectorizer.get_feature_names())
        topic_keywords = []
        for topic_weights in lda_model.components_:
            top_keyword_locs = (-topic_weights).argsort()[:n_words]
            topic_keywords.append(keywords.take(top_keyword_locs))
        return topic_keywords
        
    
    topicnames = ["Topic" + str(i) for i in range(nmf_model.n_components)]
    docnames = [i for i in range(len(data))]
    # docnames = ["Doc" + str(i) for i in range(len(data))]
    df_document_topic = pd.DataFrame(np.round(nmf_output, 2), columns=topicnames, index=docnames)
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic["dominant_topic"] = dominant_topic

    df_values = pd.DataFrame(nmf_model.components_)
    df_values.columns = list(tfidf_feature_names)
    df_values.index = topicnames

    topic_keywords = show_topics(vectorizer=tfidf_vectorizer, lda_model=nmf_model, n_words=15)
    df_keywords = pd.DataFrame(topic_keywords)
    df_keywords.columns = ['Word '+str(i) for i in range(df_keywords.shape[1])]
    df_keywords.index = ['Topic '+str(i) for i in range(df_keywords.shape[0])]

    columns = list(df_keywords.columns)
    index = list(df_keywords.index)
    df_both = pd.DataFrame(columns=columns)
    df_both['index'] = index
    df_both.set_index(['index'], drop=True, inplace=True)
    df_both = df_both.astype(object)
    for row in range(df_both.shape[0]):
        numbersRows = df_values.iloc[row,:].sort_values(ascending=False).tolist()[0:df_keywords.shape[1]]
        for column in range(len(numbersRows)):
            df_both.iat[row, column] = (df_keywords.iloc[row, column], round(numbersRows[column], 3))


CPU times: user 33.1 ms, sys: 21.9 ms, total: 55 ms
Wall time: 62.8 ms
CPU times: user 2.23 s, sys: 176 ms, total: 2.4 s
Wall time: 4.12 s
CPU times: user 37.5 ms, sys: 2.3 ms, total: 39.8 ms
Wall time: 73.4 ms


/Users/andrewwakefield2017/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 50 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"
/Users/andrewwakefield2017/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/andrewwakefield2017/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/andrewwakefield2017/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation f

In [43]:
df_doctop_20 = pd.DataFrame()

for i in range(len(df_document_topic.columns)-1):
    sorted_doctop = df_document_topic.nlargest(20, "Topic"+str(i))
    temp_list1 = list(sorted_doctop["Topic"+str(i)].index)

    temp_list2 = []

    for j in temp_list1:
        temp_list2.append(data[j])
    
    df_doctop_20["Topic"+str(i)] = temp_list2



In [46]:
df_both.to_csv('topics_and_keywords_NMF20.csv')
df_doctop_20.to_csv('Top_20_posts_per_topic_NMF20.csv')

In [ ]:
# For LDA the same process as above, only instead of NMF we used LDA package:

for n_components in range(5,45):
    
    # Build LDA model with sklearn
    lda_model = LatentDirichletAllocation(n_components=n_components, max_iter=10, random_state=100, batch_size=128, evaluate_every = -1, n_jobs = -1)
    %time lda_output = lda_model.fit_transform(data_vectorized)
    #%time lda_output2 = lda_model.fit_transform(data_vectorized2)
    LDAvis_prepared=pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, sort_topics=False)
    pyLDAvis.display(LDAvis_prepared)
    pyLDAvis.save_html(LDAvis_prepared, "lda_work"+str(n_components)+"Topics"+".html")  
    
    def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=15):
        keywords = np.array(vectorizer.get_feature_names())
        topic_keywords = []
        for topic_weights in lda_model.components_:
            top_keyword_locs = (-topic_weights).argsort()[:n_words]
            topic_keywords.append(keywords.take(top_keyword_locs))
        return topic_keywords

    
    #lda_output = lda_model.transform(data_vectorized)
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]
    docnames = ["Doc" + str(i) for i in range(len(data_lemmatized))]
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic["dominant_topic"] = dominant_topic

    df_values = pd.DataFrame(lda_model.components_)
    df_values.columns = list(vocab_d)
    df_values.index = topicnames

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=15)
    df_keywords = pd.DataFrame(topic_keywords)
    df_keywords.columns = ['Word '+str(i) for i in range(df_keywords.shape[1])]
    df_keywords.index = ['Topic '+str(i) for i in range(df_keywords.shape[0])]

    columns = list(df_keywords.columns)
    index = list(df_keywords.index)
    df_both = pd.DataFrame(columns=columns)
    df_both['index'] = index
    df_both.set_index(['index'], drop=True, inplace=True)
    df_both = df_both.astype(object)
    for row in range(df_both.shape[0]):
        numbersRows = df_values.iloc[row,:].sort_values(ascending=False).tolist()[0:df_keywords.shape[1]]
        for column in range(len(numbersRows)):
            df_both.iat[row, column] = (df_keywords.iloc[row, column], round(numbersRows[column], 3))